In [3]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.3 MB/s eta 0:00:00


In [4]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 25.7 MB/s eta 0:00:00


In [9]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.9 MB/s eta 0:00:00


# Work

In [17]:
from pdfminer.high_level import extract_text
import tabula

def extract_text_by_page(pdf_path):
    # Using extract_text from pdfminer.high_level to extract text page by page
    return [extract_text(pdf_path, page_numbers=[i]) for i in range(1, count_pages(pdf_path) + 1)]

def count_pages(pdf_path):
    # Use pdfminer to count the number of pages in the PDF
    from pdfminer.pdfpage import PDFPage
    with open(pdf_path, 'rb') as fh:
        return len(list(PDFPage.get_pages(fh)))

def find_pages_with_exact_text(pdf_path, search_texts):
    pages_with_exact_text = []
    for i, page_text in enumerate(extract_text_by_page(pdf_path), start=1):
        for search_text in search_texts:
          if search_text in page_text:  # Search for the exact phrase
            pages_with_exact_text.append(i)
    return pages_with_exact_text

pdf_path = '/content/RLI.pdf'  # Update this path to your PDF file
search_texts = ['Male Non-Smoker','Female Non-Smoker', 'Male Smoker', 'Female Smoker']  # This is the exact phrase you're looking for

# Find pages containing the exact phrase
pages_with_exact_text = find_pages_with_exact_text(pdf_path, search_texts)



In [18]:
pages_with_exact_text

[61, 62, 63, 64]

In [19]:
import pandas as pd
all_dfs = []

for i in range(len(pages_with_exact_text)):
    com_df = pd.DataFrame()
    # Extract tables from the current page
    tables = tabula.read_pdf(pdf_path, pages=pages_with_exact_text[i]+1, multiple_tables=True)
    if len(tables)>1:
      for table in  tables:
        df1 = table
        new_headers = ['Issue age'] + [str(i) for i in range(1, len(df1.columns)-1)] + ['Ultimate']
        new_headers = new_headers[:len(df1.columns)]  # Limit to the actual number of columns
        original_first_row = df1.columns
        df1.columns = new_headers
        new_row = pd.DataFrame([original_first_row], columns=new_headers)
        new_row['Issue age'].iloc[0] = float(df1['Issue age'].iloc[0])
        df = pd.concat([new_row, df1], ignore_index=True)
        com_df = pd.concat([com_df, df], ignore_index=True)
      df = com_df
    else:
      df1 = tables[0]

      new_headers = ['Issue age'] + [str(i) for i in range(1, len(df1.columns)-1)] + ['Ultimate']
      new_headers = new_headers[:len(df1.columns)]  # Limit to the actual number of columns
      original_first_row = df1.columns
      df1.columns = new_headers
      new_row = pd.DataFrame([original_first_row], columns=new_headers)
      new_row['Issue age'].iloc[0] = float(df['Issue age'].iloc[0])
      df = pd.concat([new_row, df1], ignore_index=True)
    all_dfs.append(df)

Mar 14, 2024 1:39:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 14, 2024 1:39:31 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 14, 2024 1:39:31 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 14, 2024 1:39:34 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 14, 2024 1:39:38 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 14, 2024 1:39:38 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 14, 2024 1:39:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 14, 2024 1:39:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 14, 2024 1:39:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 14, 2024 1:39:46 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 14, 2024 1:39:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 14, 2024 1:39:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [20]:
filename = 'TableData.xlsx'
with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
    # Write each DataFrame to a different sheet
    for i in range(len(all_dfs)):
      all_dfs[i].to_excel(writer, sheet_name=search_texts[i], index=False)